In [1]:
import circle_cal.model as ccm
from workalendar.usa import UnitedStates, Indiana
from workalendar.core import ChristianMixin
from datetime import datetime, date, time
from datetime import timedelta
import calendar

In [2]:
y24 = ccm.Year(2024)

In [3]:
y24.THETA_PER_DAY

0.9836065573770492

In [4]:
for m in y24:
    for d in m:
        print(d)

TypeError: 'day' is an invalid keyword argument for __new__()

In [8]:
m.part_unit()

'day'

In [9]:
for d in m:
    print(d)

(2024-12-01 00:00:00, 2024-12-31 00:00:00)


In [8]:
d + timedelta(days=31)

datetime.date(2024, 2, 1)

In [9]:
y24.len_by_days()

366.0

In [13]:
calendar.month_name[1]

'January'

In [7]:
dt = datetime(2024, 1, 1)

In [8]:
dt.year = 2025

AttributeError: attribute 'year' of 'datetime.date' objects is not writable

In [13]:
dict(**dt)

TypeError: dict() argument after ** must be a mapping, not datetime.datetime

In [1]:
d = {"a": 1}

In [6]:
d.update({"b": 2})

In [7]:
d

{'a': 1, 'b': 2}